## 7) tensor flow로 logistic regression 구현하기 - 예제 2

자, 이번에는 우리가 이전에 배운 pandas를 활용하여 

데이터를 읽고 logistic regression을 해봅시다.

우리가 오늘 진행해볼 예제 데이터는 [이녀석입니다](https://drive.google.com/open?id=1h214gzd5sTWxQ9tPR8hA8PxheMQ4lBT1)

(로컬에서 진행하실 분들은 위 링크를 클릭하여 파일을 다운받아 주세요)

이 데이터는 diabetes, 즉 당뇨에 미치는 다양한 변인들과 그 변인들에 대한 값이 나타나 있는 csv 파일로 이루어져있습니다.

우리의 목표는 다양한 변수에 특정수치가 입력될시 당뇨병인지 아닌지 분류해내는 분류기를 학습시키는 겁니다.

이진분류네요? logistic regression 갑시다!!!

### 7.1) 한글폰트설정

In [1]:
! apt-get install fonts-nanum
! sudo fc-cache -fv
! cp -rf /usr/share/fonts/truetype/nanum/Nanum* /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf/
! rm -rf /content/.cache/matplotlib/*
print('-' * 40)
print("한글폰트 설치완료: 런타임을 재실행하세요")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 6 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (5,185 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetyp

### 7.2) 구글 드라이브의 데이터 불러오기(goolge api)

In [2]:
# ------------- 데이터 입출력을 위한 google api 호출 -------------

# for google autorization
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')


# download data to python
file_id = '1h214gzd5sTWxQ9tPR8hA8PxheMQ4lBT1'
import io
from googleapiclient.http import MediaIoBaseDownload
request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  _, done = downloader.next_chunk()  
downloaded.seek(0)

print('complete')

complete


### 7.3) Import libraries

In [3]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc
rc('font',family='NanumBarunGothic')
import seaborn as sns
%matplotlib inline

print('All libraries are loaded')

All libraries are loaded


### 7.4) pandas로 데이터 불러오기

아래과정에 따라 위 예제 파일을 파싱하시오.

1. 데이터를 담을 데이터 프레임 변수명은 df로 합니다
2. csv파일이므로 read_csv를 이용합니다
3. path는 downloaded를 사용합니다(로컬의 경우 로컬 경로사용)
4. 구분자(delimiter)는 , 로 합니다
5. header는 없음으로 합니다

위 과정과 같이 파싱한후
df에서 최상단 5줄만 출력하세요

In [4]:
df = pd.read_csv(downloaded, delimiter = ',', header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,-0.294118,0.487437,0.180328,-0.292929,0.000000,0.001490,-0.531170,-0.033333,0
1,-0.882353,-0.145729,0.081967,-0.414141,0.000000,-0.207153,-0.766866,-0.666667,1
2,-0.058824,0.839196,0.049180,0.000000,0.000000,-0.305514,-0.492741,-0.633333,0
3,-0.882353,-0.105528,0.081967,-0.535354,-0.777778,-0.162444,-0.923997,0.000000,1
4,0.000000,0.376884,-0.344262,-0.292929,-0.602837,0.284650,0.887276,-0.600000,0


df의 datatype을 찍어보세요

In [5]:
df.dtypes

0    float64
1    float64
2    float64
3    float64
4    float64
5    float64
6    float64
7    float64
8      int64
dtype: object

describe명령어를 사용하여, df각 칼럼의 기술통계를 확인하세요

In [6]:
df.describe()

,0,1,2,3,4,5,6,7,8
count,759.000000,759.000000,759.000000,759.000000,759.000000,759.000000,759.000000,759.000000,759.000000
mean,-0.407657,0.218563,0.176505,-0.289735,-0.323534,-0.032245,-0.663253,-0.516162,0.653491
std,0.386260,0.306419,0.201287,0.258480,0.375544,0.205376,0.283056,0.400794,0.476171
min,-0.882353,-0.557789,-0.606557,-0.858586,-0.966903,-0.457526,-0.994876,-0.966667,0.000000
25%,-0.764706,-0.005025,0.016393,-0.494949,-0.716312,-0.178837,-0.858241,-0.866667,0.000000
50%,-0.529412,0.165829,0.180328,-0.292929,0.000000,-0.034277,-0.747225,-0.633333,1.000000
75%,0.000000,0.407035,0.311475,0.000000,0.000000,0.087928,-0.531597,-0.233333,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 7.5) 데이터 핸들링하여 Logistic regression 분류기에 집어넣고 결과 얻기

위 df의 데이터 중  0~7칼럼을 변수인 X로,

마지막 8칼럼은 예측해야할 값인 레이블인 Y로 데이터를 슬라이싱 합니다.

최종 데이터가 나와야 할 shape은 다음과 같습니다.

- X = [759, 8]

- Y = [759, 1]

shape을 바꾸고 싶으면 reshape 메써드를 호출하시면 됩니다

위 내용을 확인하여 x_data, y_data 각각을 선언하여 슬라이싱 하고 

x_data와 y_data의 shape을 출력하시오

In [7]:
x_data = df[df.columns[:8]]
y_data = df[df.columns[-1]].values.reshape(len(df),1)
print(x_data.shape, y_data.shape)

(759, 8) (759, 1)


위에서 선언한 x_data, y_data와 강의시간에 배운 logistic regression 알고리즘 코드를 참고하여, 머신러닝을 실행하고 결과를 얻으시오

In [8]:
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility


# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 8])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                       tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data})
        if step % 1000 == 0:
            print(step, cost_val)

    # Accuracy report   
    acc = sess.run(accuracy,feed_dict={X: x_data, Y: y_data})
    print("\nAccuracy: ", acc)
    
    
    
#     실제값, 예측값, 정확도 모두 다 보고싶으면 아래 실행    
#     h, c, a = sess.run([hypothesis, predicted, accuracy],
#                        feed_dict={X: x_data, Y: y_data})
#     print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)

Instructions for updating:
Colocations handled automatically by placer.
0 0.82793975
1000 0.669853
2000 0.6062456
3000 0.5665548
4000 0.54122907
5000 0.5244792
6000 0.5129687
7000 0.5047781
8000 0.49877086
9000 0.49424884
10000 0.4907668

Accuracy:  0.7628459
